In [1]:
import torch.nn
import torch.nn.functional
from torch.utils.data import Dataset, DataLoader
import torch.optim
import numpy.random
import numpy as np
import math
import pandas
import copy
import time
from sklearn.preprocessing import LabelEncoder

In [2]:
class Net(torch.nn.Module):

    def __init__(self, cat_cols, cont_cols, embeds):
        super(Net, self).__init__()
        self.cat_cols = cat_cols
        self.cont_cols = cont_cols
        self.embed = embeds
        
        #Embed the categoricals
        self.embedLayer = torch.nn.ModuleList([torch.nn.Embedding(i, 100) for i in self.embed])
        
        #normalize the numericals
        self.bn_layer = torch.nn.BatchNorm1d(len(self.cont_cols))
        
        # Linear Layers
        self.fc1 = torch.nn.Linear(len(self.embed) * 100 + len(cont_cols), 50)
        self.fc2 = torch.nn.Linear(50, 1)
    def forward(self, x):
        # Embedding Layer
        cat_encoded = [embedLayer(x[:, i+4].long()) for i, embedLayer in enumerate(self.embedLayer)]
        cat_encoded = torch.cat(cat_encoded, 1)
        cont_normalized = self.bn_layer(x[:, :4])
        x = torch.cat([cat_encoded, cont_normalized], 1)
        
        # Linear Layers
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
class GDataset(Dataset):
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        item = self.df.iloc[idx, :]
        x = item[['visitNumber', 'visitStartTime', 'hits', 'pageviews', 'newVisits', 'bounces', 
                  'channelGrouping', 'browser', 'operatingSystem', 'isMobile', 'deviceCategory', 'continent', 
                  'subContinent', 'country', 'region', 'city', 'networkDomain']].values.astype(np.float32)
        y = item[['transactionRevenue']].values.astype(np.float32)
        return {'x': torch.from_numpy(x), 'y': torch.from_numpy(y)}

In [4]:
data = pandas.read_csv("train_df.csv")

tr = {'mean': 0.22711817076655114, 'std': 2.0037093202285647} 

cont_cols = ['visitNumber', 'visitStartTime', 'hits', 'pageviews']
cat_cols = ['newVisits', 'bounces', 'channelGrouping', 'browser', 'operatingSystem', 'isMobile', 
       'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'city', 'networkDomain']

#label encode the categorical variables
label_encoders = {}
for cat_col in cat_cols:
    label_encoders[cat_col] = LabelEncoder()
    data[cat_col] = label_encoders[cat_col].fit_transform(data[cat_col])

#create testing and training set
msk = numpy.random.rand(len(data)) < 0.8
training_data = data[msk]
testing_data = data[~msk]

batch_size = 1024

train_ds = GDataset(training_data)
test_ds = GDataset(testing_data)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=8)
dataloaders = {'train': train_dl, 'val': test_dl}
dataset_sizes = {'train': len(training_data), 'val': len(testing_data)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

embeddings = [data[col].nunique() for col in cat_cols]
num_epochs=5

outputId = "fullVisitorId"
output = "transactionRevenue"
net = Net(cat_cols, cont_cols, embeddings)
model = net.to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

print(net)

/home/bread/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Net(
  (embedLayer): ModuleList(
    (0): Embedding(2, 100)
    (1): Embedding(2, 100)
    (2): Embedding(8, 100)
    (3): Embedding(54, 100)
    (4): Embedding(20, 100)
    (5): Embedding(2, 100)
    (6): Embedding(3, 100)
    (7): Embedding(6, 100)
    (8): Embedding(23, 100)
    (9): Embedding(222, 100)
    (10): Embedding(376, 100)
    (11): Embedding(649, 100)
    (12): Embedding(28064, 100)
  )
  (bn_layer): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1304, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)


In [5]:
best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 50000

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0

        # Iterate over data.
        for i, sample in enumerate(dataloaders[phase]):
            inputs, labels = sample['x'], sample['y']
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = torch.sqrt(criterion(outputs, labels))
#                 if(i % 10000 == 0):
                print("Loss at step {}: {}".format(i, loss/batch_size))

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item()

        epoch_loss = running_loss / dataset_sizes[phase]

        print('{} Loss: {:.4f}'.format(
            phase, epoch_loss))

        # deep copy the model
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())


time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Loss: {:4f}'.format(best_loss))

# load best model weights
model.load_state_dict(best_model_wts)

Epoch 0/4
----------
Loss at step 0: 0.0008147250046022236
Loss at step 1: 0.0008255068678408861
Loss at step 2: 0.0011360312346369028
Loss at step 3: 0.0010994693730026484
Loss at step 4: 0.0008290652185678482
Loss at step 5: 0.0009450821671634912
Loss at step 6: 0.0007418543100357056
Loss at step 7: 0.0010385385248810053
Loss at step 8: 0.0010465615196153522
Loss at step 9: 0.0010449700057506561
Loss at step 10: 0.0010517798364162445
Loss at step 11: 0.0009664817480370402
Loss at step 12: 0.001222106977365911
Loss at step 13: 0.0010391859104856849
Loss at step 14: 0.0010192332556471229
Loss at step 15: 0.0011897293152287602
Loss at step 16: 0.0009945209603756666
Loss at step 17: 0.0007809235830791295
Loss at step 18: 0.0008547096513211727
Loss at step 19: 0.000863875902723521
Loss at step 20: 0.0008993305964395404
Loss at step 21: 0.0010832517873495817
Loss at step 22: 0.000928671914152801
Loss at step 23: 0.0007846430526115
Loss at step 24: 0.0011296826414763927
Loss at step 25: 0.0

Loss at step 208: 0.0006938055157661438
Loss at step 209: 0.0008991117938421667
Loss at step 210: 0.0008296779706142843
Loss at step 211: 0.0007416413282044232
Loss at step 212: 0.0009733423357829452
Loss at step 213: 0.001034919056110084
Loss at step 214: 0.0009085851488634944
Loss at step 215: 0.0010549187427386642
Loss at step 216: 0.0009162560454569757
Loss at step 217: 0.0008799838833510876
Loss at step 218: 0.0009866445325314999
Loss at step 219: 0.000849766714964062
Loss at step 220: 0.001005413243547082
Loss at step 221: 0.001095828483812511
Loss at step 222: 0.0009167730459012091
Loss at step 223: 0.0009240621002390981
Loss at step 224: 0.0011656826827675104
Loss at step 225: 0.000747008016332984
Loss at step 226: 0.0009019779390655458
Loss at step 227: 0.0009533293778076768
Loss at step 228: 0.0010083012748509645
Loss at step 229: 0.0008722301572561264
Loss at step 230: 0.0009670233121141791
Loss at step 231: 0.0006777939270250499
Loss at step 232: 0.0008927374728955328
Loss 

Loss at step 416: 0.0008172695524990559
Loss at step 417: 0.0011851738672703505
Loss at step 418: 0.0009435032843612134
Loss at step 419: 0.0008730081026442349
Loss at step 420: 0.000770679849665612
Loss at step 421: 0.0010367920622229576
Loss at step 422: 0.000776427797973156
Loss at step 423: 0.0008796538459137082
Loss at step 424: 0.0009742709808051586
Loss at step 425: 0.0008425645646639168
Loss at step 426: 0.0011620365548878908
Loss at step 427: 0.0008661432657390833
Loss at step 428: 0.0009204113157466054
Loss at step 429: 0.0007510194554924965
Loss at step 430: 0.0009047690546140075
Loss at step 431: 0.0010342784225940704
Loss at step 432: 0.0009018280543386936
Loss at step 433: 0.0006372581119649112
Loss at step 434: 0.0008463160484097898
Loss at step 435: 0.0009896603878587484
Loss at step 436: 0.0009829822229221463
Loss at step 437: 0.0007752534002065659
Loss at step 438: 0.0007608502637594938
Loss at step 439: 0.0009177004103548825
Loss at step 440: 0.0010337007697671652
Lo

Loss at step 624: 0.0008276551379822195
Loss at step 625: 0.0008501489646732807
Loss at step 626: 0.0008305311785079539
Loss at step 627: 0.0007701712311245501
Loss at step 628: 0.0009136355947703123
Loss at step 629: 0.0007010410772636533
Loss at step 630: 0.001147515489719808
Loss at step 631: 0.0007109752623364329
Loss at step 632: 0.0008741256897337735
Loss at step 633: 0.0006853911909274757
Loss at step 634: 0.0009003190789371729
Loss at step 635: 0.0008213750552386045
Loss at step 636: 0.0009062248864211142
Loss at step 637: 0.0009029826032929122
Loss at step 638: 0.0009565869113430381
Loss at step 639: 0.0009173906873911619
Loss at step 640: 0.0010960923973470926
Loss at step 641: 0.001069276942871511
Loss at step 642: 0.001112470286898315
Loss at step 643: 0.0009176161256618798
Loss at step 644: 0.0008278564782813191
Loss at step 645: 0.0009136669686995447
Loss at step 646: 0.0004894525627605617
Loss at step 647: 0.0007333428366109729
Loss at step 648: 0.000978321535512805
Loss

Loss at step 126: 0.001167149981483817
Loss at step 127: 0.000803397735580802
Loss at step 128: 0.0010130482260137796
Loss at step 129: 0.0009749753517098725
Loss at step 130: 0.0009302698308601975
Loss at step 131: 0.0008698625606484711
Loss at step 132: 0.0008522705174982548
Loss at step 133: 0.0006103393970988691
Loss at step 134: 0.0008580773719586432
Loss at step 135: 0.0011689651291817427
Loss at step 136: 0.0007870314875617623
Loss at step 137: 0.0009526763460598886
Loss at step 138: 0.0007961434894241393
Loss at step 139: 0.0008532055071555078
Loss at step 140: 0.0009533194825053215
Loss at step 141: 0.0009023111779242754
Loss at step 142: 0.0011871305759996176
Loss at step 143: 0.001011616550385952
Loss at step 144: 0.0009172602440230548
Loss at step 145: 0.0009865124011412263
Loss at step 146: 0.0009527609217911959
Loss at step 147: 0.0010465746745467186
Loss at step 148: 0.0009039560682140291
Loss at step 149: 0.0009252704912796617
Loss at step 150: 0.0009194610174745321
Los

...

Loss at step 225: 0.0007559619261883199
Loss at step 226: 0.00087808130774647
Loss at step 227: 0.0007125984993763268
Loss at step 228: 0.0007763624889776111
Loss at step 229: 0.0007778374128974974
Loss at step 230: 0.0009038211428560317
Loss at step 231: 0.0008663420449011028
Loss at step 232: 0.0008484857971780002
Loss at step 233: 0.0009828782640397549
Loss at step 234: 0.0009684569085948169
Loss at step 235: 0.0009125355863943696
Loss at step 236: 0.0008499044342897832
Loss at step 237: 0.0008264556527137756
Loss at step 238: 0.0006410669884644449
Loss at step 239: 0.000967289786785841
Loss at step 240: 0.0008103570435196161
Loss at step 241: 0.0007527060806751251
Loss at step 242: 0.0007149520679377019
Loss at step 243: 0.0009818270336836576
Loss at step 244: 0.000884432636667043
Loss at step 245: 0.000662980426568538
Loss at step 246: 0.0010307440534234047
Loss at step 247: 0.000808798591606319
Loss at step 248: 0.0008162417216226459
Loss at step 249: 0.001019678427837789
Loss at

Loss at step 432: 0.0007109616999514401
Loss at step 433: 0.0006268032011575997
Loss at step 434: 0.0009301560930907726
Loss at step 435: 0.0008668942609801888
Loss at step 436: 0.0009850884089246392
Loss at step 437: 0.0006963921478018165
Loss at step 438: 0.0009024817263707519
Loss at step 439: 0.000817348132841289
Loss at step 440: 0.0008271168917417526
Loss at step 441: 0.0009429787751287222
Loss at step 442: 0.0008226028876379132
Loss at step 443: 0.0008936596568673849
Loss at step 444: 0.0008934721699915826
Loss at step 445: 0.0009751236066222191
Loss at step 446: 0.0008710070978850126
Loss at step 447: 0.0007520917570218444
Loss at step 448: 0.0008390046423301101
Loss at step 449: 0.0009344402351416647
Loss at step 450: 0.0007324186735786498
Loss at step 451: 0.000946224492508918
Loss at step 452: 0.0009323434787802398
Loss at step 453: 0.0005153972888365388
Loss at step 454: 0.0006963409250602126
Loss at step 455: 0.0008516728994436562
Loss at step 456: 0.0008437384385615587
Lo

Loss at step 641: 0.0009848763002082705
Loss at step 642: 0.0008982755825854838
Loss at step 643: 0.0008288471726700664
Loss at step 644: 0.0008148775086738169
Loss at step 645: 0.0008941202540881932
Loss at step 646: 0.0007884863298386335
Loss at step 647: 0.0007125801057554781
Loss at step 648: 0.0008281650952994823
Loss at step 649: 0.0008617826970294118
Loss at step 650: 0.0009030415676534176
Loss at step 651: 0.0008592868689447641
Loss at step 652: 0.0008880511741153896
Loss at step 653: 0.00072770711267367
Loss at step 654: 0.000946479442063719
Loss at step 655: 0.0007347139762714505
Loss at step 656: 0.000680169032420963
Loss at step 657: 0.0008559240377508104
Loss at step 658: 0.0008925978327170014
Loss at step 659: 0.0008417827193625271
Loss at step 660: 0.0006391886272467673
Loss at step 661: 0.0008307419484481215
Loss at step 662: 0.0008146024774760008
Loss at step 663: 0.0009652098524384201
Loss at step 664: 0.0008195522241294384
Loss at step 665: 0.0010080827632918954
Loss

Loss at step 144: 0.0009418336558155715
Loss at step 145: 0.001013810047879815
Loss at step 146: 0.000831460754852742
Loss at step 147: 0.0008177119889296591
Loss at step 148: 0.0007962503586895764
Loss at step 149: 0.0008107306784950197
Loss at step 150: 0.0007209171890281141
Loss at step 151: 0.0007516323821619153
Loss at step 152: 0.0008621005690656602
Loss at step 153: 0.001065160846337676
Loss at step 154: 0.000763069954700768
Loss at step 155: 0.0008157535339705646
Loss at step 156: 0.0008206847705878317
Loss at step 157: 0.0007535695331171155
Loss at step 158: 0.0008194230031222105
Loss at step 159: 0.0008756323368288577
Loss at step 160: 0.0007612524786964059
Loss at step 161: 0.0009488746291026473
Loss at step 162: 0.0007042624056339264
Loss at step 163: 0.00091220298781991
Loss at step 164: 0.0007423169445246458
Loss at step 165: 0.0010736392578110099
Loss at step 166: 0.0008508070022799075
Loss at step 167: 0.0010491885477676988
Loss at step 168: 0.000698864518199116
Loss at

NameError: name 'since' is not defined

In [ ]:
def one_batch(idx):
    sample = train_ds[idx]
    inputs, labels = sample['x'], sample['y']
    inputs.to(device)
    labels.to(device)
    outputs = model(inputs)
    loss = torch.sqrt(criterion(outputs, labels))

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f one_batch one_batch(0)

In [ ]:
# #zero the gradient buffers
# net.zero_grad()

    
# # create the optimizer
# optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()

# BATCH_SIZE = 100
# # training loop
# for epoch in range(5):
#     print("Beginning epoch ", epoch)
#     order = numpy.random.choice(training_data.shape[0], size=training_data.shape[0], replace=False)
#     for i in range(0, len(order) - BATCH_SIZE, BATCH_SIZE):
#         miniBatch = training_data.iloc[order[i:i+BATCH_SIZE]]
#         miniBatch = miniBatch.to(device)
#         optimizer.zero_grad()   # zero the gradient buffers
#         loss = torch.sqrt(criterion(net(miniBatch), torch.tensor(miniBatch[output].values, dtype=torch.float32)))
#         if(i % 10000 == 0):
#             print("Loss at step {}: {}".format(i, loss))
#         loss.backward()
#         optimizer.step()    # Does the update

# #testing loop
# output = net(testing_data)

In [ ]:
# #save output
# testing_data.loc[:,"Prediction"] = output.detach().tolist()
# testing_data.loc[:,[outputId, "Prediction"]].to_csv("output.csv")